In [1]:
%matplotlib inline
# from ExKMC.Tree import Tree # import from cloned local library followed by installing manually
import sys
sys.path.append('../')
from ExKMC_M.ExKMC.Tree import Tree

import pandas as pd
from sklearn.cluster import KMeans,DBSCAN
from sklearn.preprocessing import StandardScaler, normalize
from utils import plot_confusion_matrix,silhouette_score
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.preprocessing import LabelEncoder
from sklearn import metrics


## data preprocessing

In [3]:
# input data
DF = pd.read_csv('/home/sfy/Documents/VScodeProject/Thesis/family.csv',index_col=False)
print(f'Data shape is {DF.shape}')

grouped = DF.groupby(['family'])
# get numbers of each group 
groupCount = grouped['family'].count()
print(groupCount.sort_values(ascending=False)[:20])

# select by numbers of the top 
TOP = 11

selected = groupCount.sort_values(ascending=False)[1:TOP]

# get names
selectedNames = list(selected.index)
print(f"selected family are {selectedNames}")

# select trained
train = DF.loc[DF["family"].isin(selectedNames)]
train.reset_index(inplace=True, drop=True)
print(f"train dataset shape {train.shape}")


# encode into encoder
le = LabelEncoder()
label = le.fit_transform(train["family"])
# print(train.head(5))

# exclue name -> X_
train = train.loc[:, train.columns != "name"]
train = train.loc[:, train.columns != "family"]

# remove duplicates -> X
# parameter with first and last ? !!!! can be discussed
# X = train.drop_duplicates(keep='last')
X = train

# with X.index canbe reserved to original labels
Y = np.take(label, X.index)

print(f"Without duplications wi shape {X.shape}")
print(f"With duplications wi shape {train.shape}")

X.reset_index(inplace=True, drop=True)

# X.head(5)
# X.to_csv('testDup.csv',index=False)

Data shape is (20760, 698)
family
Airpush         6652
FakeInst        2168
Mecor           1820
Youmi           1300
Fusob           1262
Kuguo           1197
Dowgin           862
BankBot          647
Jisut            558
DroidKungFu      546
RuMMS            402
Lotoor           329
Mseg             235
Boqx             215
Minimob          203
Triada           197
Kyview           175
SlemBunk         174
SimpleLocker     172
SmsKey           165
Name: family, dtype: int64


## Train

In [19]:
# from sklearn.feature_selection import VarianceThreshold,GenericUnivariateSelect,chi2
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=0)
k = len(selectedNames)
X_train = X
X_train.shape

# success with variance
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X_train)

# Converting the numpy array into a pandas DataFrame
X_train_ = pd.DataFrame(scaled_df)

kmeans = KMeans(k,random_state=0)

kmeans.fit(X_train_)

# tree = Tree(k=k,max_leaves=8 * k)
# tree_labels = tree.fit_predict(X_train_, kmeans)

# f_a_score(Y, kmeans_labels, tree_labels=tree_labels)
kmeans.inertia_

2285167.304720277

## Evaluation

### Jaccard for label

In [7]:
# get jaccard
# get the sets index of each label kmeans predicted 
# get the sets index of true label 

for i in range(TOP):
    # index of each true
    b = np.where(Y == i ) [0]
    similarity = list()
    for family in range(TOP):
        samples = np.where(kmeans.labels_ ==family)[0]
        correctly_classified = list(set(samples).intersection(set(b)))
        union = list(set(samples).union(set(b)))
        jind = len(correctly_classified) / float(len(union))
        similarity.append(jind)
    
    print(f'--Result of family label with {i}')

    print(similarity)

# true is 0 and label is 4
# 0 -6 0.45
# 1 -0 0.63
# 4 -2 0.37
# 5 -2 0.21
# 7 -5 0.18
# 8 -2 0.3
# 9 -5 0.28

--Result of family label with 0
[0.0002830455703368242, 0.35255452644158947, 0.0014452027298273785, 0.0001503307276007216, 0.0003003003003003003, 0.0063083522583901085, 0.44898612593383136, 0.00015026296018031557, 0.0001503307276007216, 0.0014814814814814814]
--Result of family label with 1
[0.6353846153846154, 0.0, 0.03753609239653513, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 2
[0.0, 0.0027649769585253456, 0.01750913520097442, 0.0, 0.0057670126874279125, 0.07051909892262488, 0.07689116304798783, 0.0, 0.0, 0.035218783351120594]
--Result of family label with 3
[0.0, 0.001358695652173913, 0.0009432479169941833, 0.0, 0.0, 0.08974358974358974, 0.05388630272252786, 0.0, 0.0, 0.0]
--Result of family label with 4
[0.0, 0.0, 0.37244459714825634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 5
[0.0, 0.000273000273000273, 0.21659223634489866, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 6
[0.0, 0.0, 0.016411727214786488, 0.0, 0.0, 0.

### MoJofm for result

In [20]:
index_X = X_train_.index
predict = kmeans.labels_
len(predict)

# basePath = '/home/sfy/Documents/VScodeProject/Thesis'
basePath = "/home/sfy/Documents/VScodeProject/Thesis/result/"

# write distra.rsf true label
with open(basePath + os.sep + "distraTop11.rsf", "w") as f:
    for line in range(len(index_X)):
        f.write("contain " + str(Y[line]) + " " + str(line) + "\n")

# write distrb.rsf clustering result
with open(basePath + os.sep + "distrbTop11.rsf", "w") as f:
    for line in range(len(index_X)):
        f.write("contain " + str(predict[line]) + " " + str(line) + "\n")

# java -cp /home/sfy/Documents/VScodeProject/Thesis/algorithms  mojo.MoJo /home/sfy/Documents/VScodeProject/Thesis/result/distra.rsf /home/sfy/Documents/VScodeProject/Thesis/result/distrb.rsf  -fm
# top 11 data
# 80.41

### Other evaluation

In [20]:
# calculate the scores with jaccard

def f_a_score(true_labels, kmenas_labels, tree_labels):
    from sklearn.metrics import f1_score, accuracy_score

    print(f"fsocre of kmenas is {f1_score(y_true=true_labels,y_pred=kmenas_labels,average='macro')}")
    print(f"fsocre of tree score is {f1_score(y_true=true_labels,y_pred=tree_labels,average='macro')}")

    print(
        f"accuracy of kmenas is {accuracy_score(y_true=true_labels,y_pred=kmenas_labels)}"
    )
    print(
        f"accuracy of tree score is {accuracy_score(y_true=true_labels,y_pred=tree_labels)}"
    )


print(f'intra and inter distance of kmenas is {silhouette_score(X=X_train_, labels=Y)}')
# print(f'intra and inter distance of tree score is {silhouette_score(X=X_train, labels=tree_labels)}')
 

metrics.v_measure_score( Y,kmeans.labels_)
res = metrics.adjusted_rand_score(Y, kmeans.labels_)


# cost
# kmeas cost in paper: The k-means cost is the sum of squared distances of each point to the mean of points associated with the cluster.
# kmenas cost in sklearn:Opposite of the value of X on the K-means objective.
# surrogate cost:The k-means surrogate cost is the sum of squared distances of each point to the closest center of the kmeans given (or trained) in the fit method.k-means surrogate cost > k-means cost, as k-means cost is computed with respect to the optimal centers.

kmeas_cost = tree.score(X_train)
surrogate_score = tree.surrogate_score(X_train)
print(
    f"kmeans_cost is {kmeas_cost} \nsurrogate_score is {surrogate_score}\nkmenas cost in surrogate is {kmeans.score(X_train)}"
)

#test with distancs munally
# inter-distance
from scipy.spatial.distance import euclidean

dst = euclidean(kmeans.cluster_centers_[0], kmeans.cluster_centers_[1])
print(f"inter distance from kmeans:{dst}")

tree_centers = tree.get_centers()
tree_dst = euclidean(tree_centers[0], tree_centers[1])
print(f"inter distance from tree:{tree_dst}")

fsocre of kmenas is 0.07602760060373061
fsocre of tree score is 0.07051666035172852
accuracy of kmenas is 0.26633794013473155
accuracy of tree score is 0.24001666782415446
